Basic Project Outline:

1) Introduction - DONE
2) Import map data - DONE
3) Create sample of of map data - DONE
4) Basic overview of data
5) List/explain 2 or 3 problems found in data (eg. abbreviated street names, incorrect postal codes, ???)
6) Correct problem 1
7) Correct problem 2
8) Correct problem 3 (if applicable)
9) Export data as csv file
10) Import csv file into sql
11) Compute overview of staistics of file:
    a) size of the file
    b) number of unique users
    c) number of nodes and ways
    d) number of chosen type of nodes, like cafes, shops etc.
    e) Top 10 contributing users
    f) Most popular cuisines?
12) Conclusion
    a) Ideas for improving the data set more
    b) discussion about the benefits as well as some anticipated problems in implementing the improvement

<h1>Introduction</h1>

In this project I will be exploring OpenStreetMap data for Irvine, California. I chose this map data because Irvine is my current city of residence. I am already pretty familiar with the area, which will potentially help me recognize some of the bad data in the set. At the same time, I'm curious about some aspects of my city and I will use this data set to glean some new insights about it.

The data consists of XML elements called "nodes" (points of interest) and "ways" (linear features and area boundaries). Each element can have one or more tags associated with it. These tags provide more details and information about the element.  Additional information about the data structure can be found here: https://wiki.openstreetmap.org/wiki/OSM_XML

I will first explore the raw XML data using Python and audit tags that are appear to have errors, and tags that I'm interested in analyzing in more detail. Due to the scope of this project, I will not be cleaning every error I find, but will focus on two or three to meet the requirements of the project. After identifying and cleaning the problem areas, I will export the data to CSV files, then import it into a SQLite database. Finally, I will query the database to gain greater insights.



<h1>Map Area</h1>

On OpenStreetMap.org, the area I selected to include as much of Irvine I could without too much data from other cities is bounded within the following coordinates:

Longitude: -117.8792, -117.7281
Latitude: 33.7301, 33.6135

The map can be found here: https://www.openstreetmap.org/search?query=Irvine%2C%20CA#map=12/33.7299/-117.8232. The underlying data was downloaded using the Overpass API, at this link: https://overpass-api.de/api/map?bbox=-117.8792,33.6135,-117.7281,33.7301.



In [1]:
import os
os.getcwd()

'C:\\Users\\randy'

In [2]:
os.chdir("D:\Documents\School - WGU\Term 4\C750\Project")
os.getcwd()

'D:\\Documents\\School - WGU\\Term 4\\C750\\Project'

<h1>Data Exploration and Cleaning</h1>

Using the 'OSM_sampling.py' script provided by Udacity, I will create a sample file of every 100th element from the original file map data.

In [6]:
#Import osm file and create sample dataset

#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "map.osm"
SAMPLE_FILE = "sample.osm"

k = 100 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

            
with open(SAMPLE_FILE, 'wb') as output:
    output.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write(b'<osm>\n  ')
    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))
    output.write(b'</osm>')

ParseError: no element found: line 3, column 2 (<string>)

Next, I will parse the dataset to count the unique element types. 

In [ ]:
import pprint

def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename, events=('start', )):
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1
    return tags

def test():

    tags = count_tags('map.osm')
    pprint.pprint(tags)
    

if __name__ == "__main__":
    test()

 I explored these files and discovered the following problems with the data:

<ul>
<li>Abbreviated street types</li>
<li>Incomplete post codes</li>
<li>Inconsistent handling of "type" data</li>
<li>Inconsistent handling of shop data</li>
<li>Redundant data values</li>
</ul>